In [35]:
import os
import numpy as np
from simsopt.geo import BoozerSurface, SurfaceXYZTensorFourier, Volume, SurfaceRZFourier
from simsopt._core import load
from simsopt.mhd.vmec import Vmec 
from create_surface import *
from simsopt.field.wireframefield import WireframeField, enclosed_current
from simsopt.geo.wireframe import ToroidalWireframe, windowpane_wireframe

eq_name = 'wout_NAS_n2n4_AR6.2.03'  # this needs to be set based on the run
out_dir = 'Bt0.5_Bd0.5_ntf4_np8_nt12_VVa_0.28'
script_dir = os.getcwd()
eq_dir = os.path.join(script_dir, 'equilibria')
eq_name_full = os.path.join(eq_dir, eq_name + '.nc')
full_dir = os.path.join(script_dir, 'outputs', eq_name, out_dir)

dof_scale = 0.15
R0 = 1
surf_s = 1

nphi = 64
ntheta = 64
nfp = 2
nmax = 20
mmax = 20

vmec = Vmec(eq_name_full)
iota_ax = vmec.iota_axis()
iota_edge = vmec.iota_edge()
print(iota_ax)
print(iota_edge)

phis = np.linspace(0,1/nfp,nphi,endpoint=False)
thetas = np.linspace(0,1,nphi,endpoint=False)

surf_nfp1 = SurfaceRZFourier.from_wout(eq_name_full, s=surf_s, quadpoints_phi=phis,quadpoints_theta=thetas)
surf_prev = SurfaceRZFourier(mpol=surf_nfp1.mpol,ntor=surf_nfp1.ntor,nfp=2,stellsym=True,
                                quadpoints_theta=surf_nfp1.quadpoints_theta,
                                quadpoints_phi=surf_nfp1.quadpoints_phi)
surf_prev.least_squares_fit(surf_nfp1.gamma())
surf_prev.set_dofs(dof_scale*surf_prev.get_dofs())
surf_prev.set_rc(0,0,R0)

vol_target = surf_prev.volume()

0.03941123961466438
0.03054922015075745


In [41]:
import booz_xform as bx

mboz = 48 # number of poloidal harmonics for Boozer transformation
nboz = 48 # number of toroidal harmonics for Boozer transformation

b = bx.Booz_xform()
b.read_wout(eq_name_full)
b.mboz = mboz
b.nboz = nboz
b.run()
b.write_boozmn('boozmn_'+eq_name+'.nc')

bmnc = b.bmnc_b
xm = b.xm_b
xn = b.xn_b
booz_non_qs_rms = np.sqrt(np.sum(np.abs(bmnc[xn!=0])**2)/np.sum(np.abs(bmnc)**2))
booz_non_qs_max = np.max(np.abs(bmnc[xn!=0]))/np.max(np.abs(bmnc))
surf_booz = b.s_b
iota = b.iota[-1]

print('\n RMS QA metric: ',booz_non_qs_rms)
print(' Max QA metric: ',booz_non_qs_max)
print(' Axis iota: ', b.iota[0])
print(' Mean iota: ',np.mean(b.iota))
print(' Edge iota: ', b.iota[-1])

About to try reading VMEC wout file /Users/jakehalpern/Projects/C-REX/Cleaned_up_for_Github/equilibria/wout_NAS_n2n4_AR6.2.03.nc
Read ns=45, mpol=11, ntor=10, mnmax=221, mnmax_nyq=363
compute_surfs (0-based indices):  0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43
Initializing with mboz=48, nboz=48
ntheta = 194, nzeta = 194, # threads = 12
                   |        outboard (theta=0)      |      inboard (theta=pi)      |
thread js_b js zeta| |B|input  |B|Boozer    Error   | |B|input  |B|Boozer    Error |
------------------------------------------------------------------------------------
   1     4   4   0  1.560e-01  1.560e-01  4.968e-12  1.745e-01  1.745e-01  5.372e-12
                pi  1.560e-01  1.560e-01  4.966e-12  1.745e-01  1.745e-01  5.373e-12
  11    41  41   0  1.362e-01  1.362e-01  6.762e-06  1.869e-01  1.869e-01  1.338e-05
                pi  1.362e-01  1.362e-01  6.762e-06  1.869e-01  1.869e-01

In [37]:
# load the saved data from the previous run
surf_wf = load(os.path.join(full_dir, 'surf_wf.json'))
bs_tf = load(os.path.join(full_dir, 'TF_biot_savart_opt.json'))
wf_currents = np.load(os.path.join(full_dir, 'WF_currents.npy'))
loaded_dict = np.load(os.path.join(full_dir, 'WF_data.npz'))
wf_dict = {key: loaded_dict[key].item() for key in loaded_dict}
win_nPhi = wf_dict["win_nPhi"]
win_nTheta = wf_dict["win_nTheta"]
win_size = wf_dict["win_size"]
win_gap = wf_dict["win_gap"]

# Initialize an idential wireframe as the run and set the currents equal to the final currents
wf = windowpane_wireframe(surf_wf, win_nPhi, win_nTheta, win_size, win_size, \
                          win_gap, win_gap)
wf.currents = wf_currents
# get the field from the wireframe
bs_wf = WireframeField(wf)

# get the total field
bs = bs_tf + bs_wf

coils = bs_tf.coils
current_sum = sum(abs(c.current.get_value()) for c in coils)
G0 = -2. * np.pi * current_sum * (4 * np.pi * 10**(-7) / (2 * np.pi))

In [38]:
xm = [] 
xn = []

# m = 0 modes 
for n in range(0,nmax+1):
    xn.append(float(n*nfp))
    xm.append(float(0))

for m in range(1,mmax+1):
    for n in range(-nmax,nmax+1):
        xn.append(float(n*nfp))
        xm.append(float(m))

xm = np.array(xm)
xn = np.array(xn)

In [39]:
mpols = [6, 8, 10]
non_qs_rms = []
non_qs_max = []
iotas = []

for mpol in mpols:
    print(mpol)
    s = SurfaceXYZTensorFourier(
        mpol=mpol, ntor=mpol, stellsym=surf_prev.stellsym, nfp=nfp, quadpoints_phi=phis, quadpoints_theta=thetas)
    s.least_squares_fit(surf_prev.gamma())
    
    vol = Volume(s)
    constraint_weight = 1
    boozer_surface = BoozerSurface(bs, s, vol, vol_target, constraint_weight)
    # second derivatives not implemented in wireframe yet (as of 10/3/2024), so only run L-BFGS, not Newton
    # Newton would be run if you use run_code instance
    res = boozer_surface.minimize_boozer_penalty_constraints_LBFGS(constraint_weight=constraint_weight, iota=iota, G=G0, \
            tol=1e-8, maxiter=1500, verbose=True, limited_memory=False, weight_inv_modB=True)
    iotas.append(res['iota'])
    print('iota: ', iotas[-1])

    x = s.gamma() 
    x = x.reshape((x.size//3, 3)).copy()
    bs.set_points(x)
    B = bs.B()
    modB = np.sqrt(np.sum(B*B,axis=-1)).reshape(np.shape(s.gamma()[:,:,0]))
    
    theta, phi = np.meshgrid(s.quadpoints_theta, s.quadpoints_phi)		
    
    phi *= 2*np.pi 
    theta *= 2*np.pi 
    
    bmnc = np.zeros_like(xm)
    for im in range(len(xm)):
    	angle = xm[im] * theta  - xn[im] * phi
    	bmnc[im] = np.sum(modB*np.cos(angle))/np.sum(np.cos(angle)**2)

    modB_ift = np.zeros_like(modB)
    for im in range(len(xm)):
    	angle = xm[im] * theta  - xn[im] * phi
    	modB_ift += bmnc[im] * np.cos(angle)

    non_qs_rms.append(np.sqrt(np.sum(np.abs(bmnc[xn!=0])**2)/np.sum(np.abs(bmnc)**2)))
    non_qs_max.append(np.max(np.abs(bmnc[xn!=0]))/np.max(np.abs(bmnc)))

    print('non_qs_rms: ',non_qs_rms[-1])
    print('non_qs_max: ',non_qs_max[-1])
    
    s_prev = s

6
0.009456682072644077
51.033143581887245
0.005430830343790934
0.009782812704953424
0.004108050151745889
0.004365274109796527
0.00349892207052595
0.0028413976138075762
0.0022796189831071245
0.0020283374336343856
0.0021160552257224337
0.001920214542070638
0.0017764482608196698
0.0018881992491524499
0.0017240560483782039
0.0016844353542353715
0.0016406865411640213
0.0015951892931634329
0.0015388801794331028
0.0015222481865787786
0.0015044934131769095
0.0014804762174830597
0.0014421461089488474
0.0013916819844325687
0.0013487033874202293
0.0012862940868545397
0.0011974684114399224
0.0011162460154186859
0.0010984322797790286
0.0010722544013782274
0.0010465869504183599
0.0010192306801654941
0.0009919319122916137
0.0009921595947252585
0.0009782097837929837
0.0009734982936700904
0.0009664552997177603
0.0009629223493548382
0.0009590403292862872
0.0009542763025585768
0.0009486931685254029
0.0009414919276144209
0.0009334934389402088
0.0009264294712311349
0.0009172077277296473
0.00090675464153259

In [43]:
lines = [f"Iota Values: \n"
         f"From VMEC: iota_edge       = {iota_edge:.5f}, iota_axis = {iota_ax:.5f} \n", 
         f"From booz xform: iota_edge = {b.iota[-1]:.5f}, iota_axis = {b.iota[0]:.5f}, iota_mean = {np.mean(b.iota):.5f} \n",
         f"\n"
         f"QS Error Values \n",
         f"From booz xform: rms QS metric = {booz_non_qs_rms:.5f}, max QS metric = {booz_non_qs_max:.5f}\n", \
         f"From Boozer Surface: \n", 
         f"mpols = {mpols} \n", 
         f"rms QS metrics = {non_qs_rms} \n", 
         f"max QS metrics = {non_qs_max} \n"]

with open(os.path.join(full_dir, "qs_error.txt"), "w") as file1:
    file1.writelines(lines)